In [1]:
import stripe
import pandas as pd
import numpy as np
import psycopg2
from time import sleep
from config import api_key
from config import host, database, user, password
from queries import *
import pandas.io.sql as sqlio


In [2]:
def get_df():
    df = pd.DataFrame()
    stripe.api_key = api_key
                                                        
    #first query to get response id
    first_response = stripe.Subscription.list(status='all')
    #creating dataframe with responses
    df = df.append(pd.json_normalize(first_response['data']))
    #defining next page for loops
    start_after = first_response['data'][-1]['id']
    print('Getting Results')

    while True:    
        try:
            #appending to the dataframe
            response = stripe.Subscription.list(starting_after=start_after, status='all')
            subscription_df = pd.json_normalize(response['data'])
            df = df.append(subscription_df, ignore_index=True)
            #giving the key for the next iteration
            start_after = response['data'][-1]['id']
            #ask george about rate limit issues
            sleep(.1)
        except:
            print('Stripe Request went wrong')
        
        
        if response["has_more"] == False:
            break
    return df


In [3]:
df = get_df()
df

Getting Results


,id,object,application,application_fee_percent,billing_cycle_anchor,billing_thresholds,cancel_at,cancel_at_period_end,canceled_at,collection_method,...,plan.livemode,plan.nickname,plan.product,plan.tiers,plan.tiers_mode,plan.transform_usage,plan.trial_period_days,plan.usage_type,pause_collection.behavior,pause_collection.resumes_at
0,sub_1L8hAPLnLiSMHjeFOk7Hzzf0,subscription,None,None,1654764997,None,NaN,False,NaN,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
1,sub_1L8bGPLnLiSMHjeFCN2KrbKW,subscription,None,None,1654742305,None,NaN,False,NaN,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
2,sub_1L8agULnLiSMHjeFhQcBmnmh,subscription,None,None,1654740078,None,NaN,False,NaN,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
3,sub_1L8Y24LnLiSMHjeFnJm7oAZ7,subscription,None,None,1654729884,None,NaN,False,NaN,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
4,sub_1L8V3cLnLiSMHjeFfbKghszO,subscription,None,None,1654718448,None,NaN,False,NaN,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,sub_HxRsSFzSCdOCao,subscription,None,None,1599195439,None,None,False,1608583771.0,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
1268,sub_Hx4l1OOH4jFpCg,subscription,None,None,1599109429,None,None,False,1606172232.0,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
1269,sub_Hx4hMQdjs9NK8q,subscription,None,None,1599109203,None,None,False,1599109376.0,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN
1270,sub_Hww73rW1qJJn9n,subscription,None,None,1599077260,None,None,False,1599082600,charge_automatically,...,True,None,prod_Hj1soBH2jO8V4b,None,None,None,None,licensed,NaN,NaN


In [4]:
def cleaning_df(df):
    df = df[['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end', 'canceled_at', 'created', 'current_period_end', 'current_period_start', 'customer', 'ended_at', 'quantity', 'start_date', 'status']]
    df['billing_cycle_anchor'] = pd.to_datetime(df['billing_cycle_anchor'], unit='s')
    df['canceled_at'] = pd.to_datetime(df['canceled_at'], unit='s')
    df['cancel_at'] = pd.to_datetime(df['cancel_at'], unit='s')
    df['created'] = pd.to_datetime(df['created'], unit='s')
    df['current_period_end'] = pd.to_datetime(df['current_period_end'], unit='s')
    df['current_period_start'] = pd.to_datetime(df['current_period_start'], unit='s')
    df['ended_at'] = pd.to_datetime(df['ended_at'], unit='s')
    df['start_date'] = pd.to_datetime(df['start_date'], unit='s')
    df = df.replace({np.NaN: None})
    return df


In [5]:
df = cleaning_df(df)
df

C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_11080/3571459582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billing_cycle_anchor'] = pd.to_datetime(df['billing_cycle_anchor'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_11080/3571459582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['canceled_at'] = pd.to_datetime(df['canceled_at'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_11080/3571459582.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer,ended_at,quantity,start_date,status
0,sub_1L8hAPLnLiSMHjeFOk7Hzzf0,2022-06-09 08:56:37,None,False,None,2022-06-09 08:56:37,2022-07-09 08:56:37,2022-06-09 08:56:37,cus_Kq5dUDEXGqYNFX,None,1,2022-06-09 08:56:37,active
1,sub_1L8bGPLnLiSMHjeFCN2KrbKW,2022-06-09 02:38:25,None,False,None,2022-06-09 02:38:25,2022-07-09 02:38:25,2022-06-09 02:38:25,cus_LqFdFeIJQRC9be,None,1,2022-06-09 02:38:25,active
2,sub_1L8agULnLiSMHjeFhQcBmnmh,2022-06-09 02:01:18,None,False,None,2022-06-09 02:01:18,2022-07-09 02:01:18,2022-06-09 02:01:18,cus_Lma7BlsNPcylAP,None,1,2022-06-09 02:01:18,active
3,sub_1L8Y24LnLiSMHjeFnJm7oAZ7,2022-06-08 23:11:24,None,False,None,2022-06-08 23:11:24,2022-07-08 23:11:24,2022-06-08 23:11:24,cus_LdsM14Xsj9KT48,None,1,2022-06-08 23:11:24,active
4,sub_1L8V3cLnLiSMHjeFfbKghszO,2022-06-08 20:00:48,None,False,None,2022-06-08 20:00:48,2022-07-08 20:00:48,2022-06-08 20:00:48,cus_LqBPgyw48imqA0,None,1,2022-06-08 20:00:48,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,sub_HxRsSFzSCdOCao,2020-09-04 04:57:19,None,False,2020-12-21 20:49:31,2020-09-04 04:57:19,2021-01-04 04:57:19,2020-12-04 04:57:19,cus_HsxJ4mZ2mIvuaw,2020-12-21 20:49:31,1,2020-09-04 04:57:19,canceled
1268,sub_Hx4l1OOH4jFpCg,2020-09-03 05:03:49,None,False,2020-11-23 22:57:12,2020-09-03 05:03:49,2020-12-03 05:03:49,2020-11-03 05:03:49,cus_HsRzIRAfM5a8Pt,2020-11-23 22:57:12,1,2020-09-03 05:03:49,canceled
1269,sub_Hx4hMQdjs9NK8q,2020-09-03 05:00:03,None,False,2020-09-03 05:02:56,2020-09-03 05:00:03,2020-10-03 05:00:03,2020-09-03 05:00:03,cus_HsRzIRAfM5a8Pt,2020-09-03 05:02:56,1,2020-09-03 05:00:03,canceled
1270,sub_Hww73rW1qJJn9n,2020-09-02 20:07:40,None,False,2020-09-02 21:36:40,2020-09-02 20:07:40,2020-10-02 20:07:40,2020-09-02 20:07:40,cus_HsRzIRAfM5a8Pt,2020-09-02 21:36:40,1,2020-09-02 20:07:40,canceled


In [6]:
def connect(host, database, user, password):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host, database=database, user=user, password=password)
        conn.autocommit = True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(host, database, user, password)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
def setup(conn, table):
    cur = conn.cursor()
    cur.execute(table)
    

In [8]:
setup(conn, create_subscription_table)

In [10]:
df.columns

Index(['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end',
       'canceled_at', 'created', 'current_period_end', 'current_period_start',
       'customer', 'ended_at', 'quantity', 'start_date', 'status'],
      dtype='object')

In [13]:
def insert_values(conn, df):
    
    cur = conn.cursor()
    
    for value in df.values:
        subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, quantity, start_date, status = value
        #insert subscriber record
        subscriber_data = (subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at,quantity, start_date, status)
        cur.execute(insert_table, subscriber_data)
    
    conn.commit()


In [14]:
insert_values(conn, df)

In [15]:
test_df_4_me = sqlio.read_sql_query("SELECT * FROM subscriptions", conn)
test_df_4_me

,subscription_id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer_id,ended_at,quantity,start_date,status
0,sub_1L3DMwLnLiSMHjeF0Ma0xP40,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_Lkil0CHKttza9s,None,1,2022-05-25,active
1,sub_1L3BJCLnLiSMHjeFdcn3bqz6,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_LkgZyDBSqAJuzr,None,1,2022-05-25,active
2,sub_1L396ALnLiSMHjeFsXIV8dRp,2022-05-25,None,False,None,2022-05-25,2022-06-25,2022-05-25,cus_LkeLbjrx4lbFMQ,None,1,2022-05-25,active
3,sub_1L393zLnLiSMHjeFm02EDpKO,2022-05-25,None,False,2022-05-26,2022-05-25,2022-06-25,2022-05-25,cus_LkeLbjrx4lbFMQ,2022-05-26,1,2022-05-25,incomplete_expired
4,sub_1L393YLnLiSMHjeFClYhqQcR,2022-05-25,None,False,2022-05-26,2022-05-25,2022-06-25,2022-05-25,cus_LkeLbjrx4lbFMQ,2022-05-26,1,2022-05-25,incomplete_expired
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,sub_1KCqRELnLiSMHjeFGfnTUIVJ,2021-12-31,None,False,2022-02-21,2021-12-31,2022-02-28,2022-01-31,cus_Ksba44uLMiLqCt,2022-02-21,1,2021-12-31,canceled
1268,sub_1KCcr3LnLiSMHjeFj2RyJjlB,2021-12-31,None,False,2022-03-21,2021-12-31,2022-03-31,2022-02-28,cus_KsNagZwJCUrVE9,2022-03-21,1,2021-12-31,canceled
1269,sub_1KCYzrLnLiSMHjeFmveyclqk,2021-12-31,None,False,2022-05-21,2021-12-31,2022-05-31,2022-04-30,cus_KsJUk6dRN4pUer,2022-05-21,1,2021-12-31,canceled
1270,sub_1KCXoRLnLiSMHjeF7Bnhiqz2,2021-12-30,None,False,None,2021-12-30,2022-06-30,2022-05-30,cus_Kr66P7B4YDO8m4,None,1,2021-12-30,active


In [16]:
conn.close()